In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool


llm = ChatOpenAI(model="gpt-3.5-turbo-0125")


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]



In [2]:
tools

[StructuredTool(name='add', description='Adds a and b.\n\n    Args:\n        a: first int\n        b: second int', args_schema=<class 'pydantic.v1.main.addSchema'>, func=<function add at 0x10bbdd240>),
 StructuredTool(name='multiply', description='Multiplies a and b.\n\n    Args:\n        a: first int\n        b: second int', args_schema=<class 'pydantic.v1.main.multiplySchema'>, func=<function multiply at 0x10bbdd2d0>)]

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [add, multiply]

In [4]:
tools

[__main__.add, __main__.multiply]

In [6]:
llm_with_tools = llm.bind_tools(tools, tool_choice='auto')

# Binding tool schemas



In [8]:
query = "What is 3 * 12? Also, what is 11 + 49?"


result = llm_with_tools.invoke(query).tool_calls
result



[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_6jOu2h5CpBIns158ZYzZ2seS',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_SV0mIK2xGXfWr3MxclsSzBVZ',
  'type': 'tool_call'}]

In [9]:
type(result)

list

In [12]:
query = "What is 3*20"


result = llm_with_tools.invoke(query).tool_calls
result

[{'name': 'multiply',
  'args': {'a': 3, 'b': 20},
  'id': 'call_gONDcLYq9hNmW3ztmdNgiAIv',
  'type': 'tool_call'}]

In [13]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[multiply, add])
chain.invoke(query)

[multiply(a=3, b=20)]

In [14]:
query = "What is 3 * 12? Also, what is 11 + 49?"
result = chain.invoke(query)
result


[multiply(a=3, b=12), add(a=11, b=49)]

In [19]:
result[0].

multiply(a=3, b=12)

In [20]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser




chain = llm_with_tools | JsonOutputKeyToolsParser()
chain.invoke(query)

ValidationError: 1 validation error for JsonOutputKeyToolsParser
key_name
  field required (type=value_error.missing)

## One way to create the tool

- Define the schema
- Use the ChatPromptTemplate
- Define 3 tools with their schemas
- Select the tool_choice='auto' in the ChatOpenAI class

In [53]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.runnables import RunnablePassthrough
from pprint import pprint


class ShowHotel(BaseModel):
    route: list[str] = Field(..., description="list of locations to visit")

def get_list_hotels(route: list):
    return True

class ShowMap(BaseModel):
    route: list[str] = Field(..., description="list of locations to visit")
    
def get_map(route:list):
    return True

class ShowHotelDetails(BaseModel):
    hotel_name: str = Field(..., description="a name of hotel")
    
def get_hotel_detail(hotel_name:str):
    return True

# define tools from functions
list_hotel_tool = StructuredTool.from_function(
    func=get_list_hotels,
    name="Get_List_Hotels",
    description="useful for when you need to show the list of hotels in a route",
    args_schema=ShowHotel,
    return_direct=False
)

hotel_detail_tool = StructuredTool.from_function(
    func=get_hotel_detail,
    name="Get_Hotel_Details",
    description="useful for when you need to show the details of a hotel",
    args_schema=ShowHotelDetails,
    return_direct=False
)


map_tool = StructuredTool.from_function(
    func=get_map,
    name="Get_Map",
    description="useful for when you need to show the map of a route",
    args_schema=ShowMap,
    return_direct=False
)


tools = [list_hotel_tool, map_tool, hotel_detail_tool]

# ! Demonstrate in the next section
# messages placeholders 
# Prompt template that assumes variable is already list of messages.
# A placeholder which can be used to pass in a list of messages.
# use for the chat history

# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are a helpful assistant",
#         ),
#         MessagesPlaceholder(variable_name="messages", optional=True),
#     ]
# )

prompt =  ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant",
        ),
        ("human", "{question}")
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True)


# Create the tools to bind to the model
tools = [convert_to_openai_function(t) for t in tools]

# MODFIICATION: we're using bind_tools instead of bind_function
# model = {"messages": RunnablePassthrough()} | prompt | llm.bind_tools(tools)
model = prompt | llm.bind_tools(tools, tool_choice='auto')


In [54]:
response = model.invoke({"question": "show maps for route Phú Quốc - Bình Thuận - Quảng Ninh"})
print(response)
pprint(response.tool_calls)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_CdH3FWs1eelB9xBDBqDqZXvq', 'function': {'arguments': '{"route": ["Phú Quốc", "Bình Thuận", "Quảng Ninh"]}', 'name': 'Get_Map'}, 'type': 'function'}]} response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_b591f37d7c'} id='run-e516f87b-cda1-4787-aee1-7b05bf35ec29-0' tool_calls=[{'name': 'Get_Map', 'args': {'route': ['Phú Quốc', 'Bình Thuận', 'Quảng Ninh']}, 'id': 'call_CdH3FWs1eelB9xBDBqDqZXvq', 'type': 'tool_call'}]
[{'args': {'route': ['Phú Quốc', 'Bình Thuận', 'Quảng Ninh']},
  'id': 'call_CdH3FWs1eelB9xBDBqDqZXvq',
  'name': 'Get_Map',
  'type': 'tool_call'}]


In [55]:
response = model.invoke({"question": "danh sách khách sạn  Phú Quốc - Bình Thuận - Quảng Ninh"})
print(response)
pprint(response.tool_calls)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_YnXDESroe7kROpBxp1rw4t1r', 'function': {'arguments': '{"route":["Phú Quốc","Bình Thuận","Quảng Ninh"]}', 'name': 'Get_List_Hotels'}, 'type': 'function'}]} response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_b591f37d7c'} id='run-81807869-cb43-41c5-8e40-85c53064d4d1-0' tool_calls=[{'name': 'Get_List_Hotels', 'args': {'route': ['Phú Quốc', 'Bình Thuận', 'Quảng Ninh']}, 'id': 'call_YnXDESroe7kROpBxp1rw4t1r', 'type': 'tool_call'}]
[{'args': {'route': ['Phú Quốc', 'Bình Thuận', 'Quảng Ninh']},
  'id': 'call_YnXDESroe7kROpBxp1rw4t1r',
  'name': 'Get_List_Hotels',
  'type': 'tool_call'}]


In [56]:
response = model.invoke({"question": "thông tin chi tiết khách sạn Mường Thanh"})
print(response)
pprint(response.tool_calls)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_jqXWabNAwLrYe3yJn5j4xzlj', 'function': {'arguments': '{"hotel_name":"Mường Thanh"}', 'name': 'Get_Hotel_Details'}, 'type': 'function'}]} response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_b591f37d7c'} id='run-879b8d57-d9d4-49f1-a2bf-877a27b2a46f-0' tool_calls=[{'name': 'Get_Hotel_Details', 'args': {'hotel_name': 'Mường Thanh'}, 'id': 'call_jqXWabNAwLrYe3yJn5j4xzlj', 'type': 'tool_call'}]
[{'args': {'hotel_name': 'Mường Thanh'},
  'id': 'call_jqXWabNAwLrYe3yJn5j4xzlj',
  'name': 'Get_Hotel_Details',
  'type': 'tool_call'}]


## Second way to create the tool

Useful when receive the list of messages from user 

- Define the schema
- Use the ChatPromptTemplate
- Define 3 tools with their schemas
- Select the tool_choice='auto' in the ChatOpenAI class
- Use MessagePlaceHolder and PassThroughRunnable


MessagePlaceHolder -> optional=True => Not error

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.runnables import RunnablePassthrough
from pprint import pprint


class ShowHotel(BaseModel):
    route: list[str] = Field(..., description="list of locations to visit")

def get_list_hotels(route: list):
    return True

class ShowMap(BaseModel):
    route: list[str] = Field(..., description="list of locations to visit")
    
def get_map(route:list):
    return True

class ShowHotelDetails(BaseModel):
    hotel_name: str = Field(..., description="a name of hotel")
    
def get_hotel_detail(hotel_name:str):
    return True

# define tools from functions
list_hotel_tool = StructuredTool.from_function(
    func=get_list_hotels,
    name="Get_List_Hotels",
    description="useful for when you need to show the list of hotels in a route",
    args_schema=ShowHotel,
    return_direct=False
)

hotel_detail_tool = StructuredTool.from_function(
    func=get_hotel_detail,
    name="Get_Hotel_Details",
    description="useful for when you need to show the details of a hotel",
    args_schema=ShowHotelDetails,
    return_direct=False
)


map_tool = StructuredTool.from_function(
    func=get_map,
    name="Get_Map",
    description="useful for when you need to show the map of a route",
    args_schema=ShowMap,
    return_direct=False
)


tools = [list_hotel_tool, map_tool, hotel_detail_tool]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. You may not need to use tools for every query - the user may just want to chat!",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
        ("human", "{question}")  # This is where you would add the user's input
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True)


# Create the tools to bind to the model
tools = [convert_to_openai_function(t) for t in tools]

# MODFIICATION: we're using bind_tools instead of bind_function
# model = {"messages": RunnablePassthrough(), "question": "Hello world"} | prompt | llm.bind_tools(tools)

# prompt.format_messages(messages=[("human", "show maps for route Phú Quốc - Bình Thu")])
model = prompt | llm.bind_tools(tools, tool_choice='auto')



Model will response incorrectly, because it doesnot have the historical data

In [7]:
response = model.invoke({"question": "danh sách khách sạn của cả hành trình trên"})
print(response)
pprint(response.tool_calls)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_sCh4vD2JEyTnGa9LEVWYYNw3', 'function': {'arguments': '{"route":["Hanoi","Hue","Danang","Nha Trang","Ho Chi Minh City"]}', 'name': 'Get_List_Hotels'}, 'type': 'function'}]} response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_b591f37d7c'} id='run-2b65d77a-abe0-47f6-b37d-497d40051c7f-0' tool_calls=[{'name': 'Get_List_Hotels', 'args': {'route': ['Hanoi', 'Hue', 'Danang', 'Nha Trang', 'Ho Chi Minh City']}, 'id': 'call_sCh4vD2JEyTnGa9LEVWYYNw3', 'type': 'tool_call'}]
[{'args': {'route': ['Hanoi',
                     'Hue',
                     'Danang',
                     'Nha Trang',
                     'Ho Chi Minh City']},
  'id': 'call_sCh4vD2JEyTnGa9LEVWYYNw3',
  'name': 'Get_List_Hotels',
  'type': 'tool_call'}]


Add the historical data -> Model will response correctly

In [8]:
response = model.invoke({"messages": [("human", "Tôi muốn chương trình du lịch Phú Quốc - Bình Thuận - Quảng Ninh trong 5 ngày/4 đêm, khách sạn 4 sao?")], "question": "danh sách khách sạn của cả hành trình"})
print(response)
pprint(response.tool_calls)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_PaXoP1LHF03gCjqMnLYL3sF6', 'function': {'arguments': '{"route":["Phu Quoc","Binh Thuan","Quang Ninh"]}', 'name': 'Get_List_Hotels'}, 'type': 'function'}]} response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_b591f37d7c'} id='run-eb5ecfd9-a47f-4007-97a2-8b2487f901a7-0' tool_calls=[{'name': 'Get_List_Hotels', 'args': {'route': ['Phu Quoc', 'Binh Thuan', 'Quang Ninh']}, 'id': 'call_PaXoP1LHF03gCjqMnLYL3sF6', 'type': 'tool_call'}]
[{'args': {'route': ['Phu Quoc', 'Binh Thuan', 'Quang Ninh']},
  'id': 'call_PaXoP1LHF03gCjqMnLYL3sF6',
  'name': 'Get_List_Hotels',
  'type': 'tool_call'}]


In [18]:
import json
import os
import requests
import ast
import json
from dotenv import load_dotenv

load_dotenv()

BEAR_TOKEN_API_KEY = os.getenv("BEAR_TOKEN_API_KEY")

DEBUG = True

if DEBUG:
    url = "http://103.163.25.97:8002/v1/chat/completions"
else:
    url = "https://api.mistral.ai/v1/chat/completions"


data = {
    "model": "mistral-small-latest",
    "messages": [{
        "role": "user",
        "content": "list all the status of devices in kitchen"
    }],
    "tools": [{
                "type": "function",
                "function": {
                    "name": "list_device_status_by_zone",
                    "description": "List the status of devices in a specific zone.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "zone": {
                                "type": "string",
                                "description": "The zone to list the device status for. Can be 'kitchen' or 'outdoor'."
                            }
                        },
                        "required": ["zone"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "list_available_zones",
                    "description": "List the available zones of the house.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            }],
    "tool_choice": "auto",
    "stream": True
}

headers = {"Content-type": "application/json", "Authorization": f"Bearer {BEAR_TOKEN_API_KEY}"}

# print chunk
# with requests.post(url, data=json.dumps(data), headers=headers, stream=True) as r:
#     for chunk in r.iter_content(1024):
#         print(chunk)
#         print("\n\n")

# # write the streaming to terminal
# with requests.post(url, data=json.dumps(data), headers=headers, stream=True) as r:
#     for chunk in r.iter_content(1024):
#         # print(chunk.decode().split("data: ")[1])
#         response_dict = json.loads(chunk.decode().split("data: ")[1])
#         content = response_dict["choices"][0]["delta"]["content"]

#         print(content, end="")


with requests.post(url, data=json.dumps(data), headers=headers, stream=True) as r:
    for chunk in r.iter_content(1024):
        print(chunk, "")


b'data: {"id": "644dc7d360bc472392e9f7863ebc7aba", "object": "chat.completion.chunk", "created": 3283120793813, "model": "mistral-medium-latest", "choices": [{"index": 0, "delta": {"tool_calls": null}, "finish_reason": null, "logprobs": null}]}\n\n' 
b'data: {"id": "644dc7d360bc472392e9f7863ebc7aba", "object": "chat.completion.chunk", "created": 3283120793813, "model": "mistral-medium-latest", "choices": [{"index": 0, "delta": {"tool_calls": [{"id": "7GWiYgE2d", "function": {"name": "showHotels", "arguments": "[\'Ph\\u00fa Qu\\u1ed1c\', \'B\\u00ecnh Thu\\u1eadn\', \'Qu\\u1ea3ng Ninh\']"}}]}, "finish_reason": "tool_calls", "logprobs": null}]}\n\n' 
b'data: [DONE]\n\n' 


In [22]:
import json
import os
import requests
import ast
import json
from dotenv import load_dotenv

load_dotenv()

BEAR_TOKEN_API_KEY = os.getenv("BEAR_TOKEN_API_KEY")

DEBUG = True

if DEBUG:
    url = "http://103.163.25.97:8002/v1/chat/completions"
else:
    url = "https://api.mistral.ai/v1/chat/completions"


data = {
    "model": "mistral-small-latest",
    "messages": [{
        "role": "user",
        "content": "list all the status of devices in kitchen"
    }],
    "tools": [{
                "type": "function",
                "function": {
                    "name": "list_device_status_by_zone",
                    "description": "List the status of devices in a specific zone.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "zone": {
                                "type": "string",
                                "description": "The zone to list the device status for. Can be 'kitchen' or 'outdoor'."
                            }
                        },
                        "required": ["zone"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "list_available_zones",
                    "description": "List the available zones of the house.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            }],
    "tool_choice": "auto",
    "stream": True
}

headers = {"Content-type": "application/json", "Authorization": f"Bearer {BEAR_TOKEN_API_KEY}"}

# print chunk
# with requests.post(url, data=json.dumps(data), headers=headers, stream=True) as r:
#     for chunk in r.iter_content(1024):
#         print(chunk)
#         print("\n\n")

# # write the streaming to terminal
# with requests.post(url, data=json.dumps(data), headers=headers, stream=True) as r:
#     for chunk in r.iter_content(1024):
#         # print(chunk.decode().split("data: ")[1])
#         response_dict = json.loads(chunk.decode().split("data: ")[1])
#         content = response_dict["choices"][0]["delta"]["content"]

#         print(content, end="")


with requests.post(url, data=json.dumps(data), headers=headers, stream=True) as r:
    for chunk in r.iter_content(1024):
        print(chunk, "")


b'data: {"id": "74d95cd40aa44baab81f533a66d18ade", "object": "chat.completion.chunk", "created": 1722828204, "model": "mistral-large-latest", "choices": [{"index": 0, "delta": {"content": null, "tool_calls": [{"id": "gJDV2w6fE", "function": {"name": "showHotels", "arguments": "{\\"route\\": [\\"Hanoi\\", \\"Hue\\", \\"Da Nang\\", \\"Hoi An\\", \\"Nha Trang\\", \\"Ho Chi Minh\\"]}"}}]}, "finish_reason": "tool_calls", "logprobs": null}], "usage": {"prompt_tokens": 433, "total_tokens": 483, "completion_tokens": 50}}\n\ndata: [DONE]\n\n' 
